# Importing the Data

In [49]:
# to handle datasets
import pandas as pd

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)
pd.pandas.set_option('display.max_rows', 100)

In [50]:
# load the csvs - it is available open source and online
df = pd.read_csv('2019_Winter_Data_Science_Intern_Challenge_Data_Set.csv')
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


In [51]:
# order id can be dropped since theres only unique values
df['order_id'].value_counts().value_counts()

1    5000
Name: order_id, dtype: int64

# App Question

Question 1: Given some sample data, write a program to answer the following

On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 
What metric would you report for this dataset?
What is its value?

# App Answer

One quick and easy solution is to find the median instead of the mean so that it's easier to ignore the outliers. I found the **median to be $284**, which is a much more believable amount.

Another quick solution is to find the outliers for the order amount, and **create a new dataset with the outliers removed since the outliers** are huge in value and skew the AOV to be much higher. After removing the outliers, we found the **average (mean) order amount to be ~$294**. To note, the median didn't change much with the median order amount being $280. Therefore I'm pretty confident using the **$294 as the average order amount**.

However, in this context, we're not exactly sure what insight the client wants to glean from the AOV (normally I would clear these details with the client in a professional setting), so there are other solutions that could help client gain insight about the AOV for these sneaker stores. Since each store only sells one product, it might make sense to calculate the average order value for each store individually. **The csv 'shopify_per_store_aov' in the Github repo shows the per store AOV**, as well as other stats that help provide the client a more granular picture of the AOV.

Although removing outliers is a quick and easy method to "fix" the AOV, it's important to look at the outliers and see if it makes sense to remove them. For example, for shop 42, at a quick glance it's easy to see that there's multiple $704,000 orders, greatly skewing the AOV. But after taking a deeper dive, we see that shop 42 sells sneakers for $352 per pair. And although the amount isn't egregious, there is one customer (user 607) who bought 2000 pairs per order for 17 orders throughout the month. As each order is $704,000, it greatly skews the AOV for all the sneaker stores in the data. We should then try to see if we can find any more info about the user (are they a warehouse stocking inventory, a rich person with an unhealthy obsession, or is there a system error that's recording incorrect information).

Also another outlier was shop 78 who's product sells for $25725. Seeing how 88 pairs were sold, with an average of ~$49000 per order, it greatly skews the overall AOV. I would recommend looking into what exactly the product was to make sure that there wasn't an error in recording the sneaker selling price.

# Data Exploration

In [52]:
def wrangle(path):
    df = pd.read_csv(path)

    # convert created_at column to a datetime object
    df['created_at']= pd.to_datetime(df['created_at'])

    # sort the dataframe by date
    df = df.sort_values(by=['created_at'])

    # drop high-cardinality and irrelevant features
    df.drop('order_id', axis=1, inplace=True)

    return df

In [53]:
path = '2019_Winter_Data_Science_Intern_Challenge_Data_Set.csv'
df = wrangle(path)
df.head()

,shop_id,user_id,order_amount,total_items,payment_method,created_at
1862,39,738,536,4,cash,2017-03-01 00:08:09
1741,39,910,268,2,cash,2017-03-01 00:10:19
3228,97,912,324,2,cash,2017-03-01 00:14:12
1267,80,798,290,2,credit_card,2017-03-01 00:19:31
2689,49,799,258,2,credit_card,2017-03-01 00:22:25


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 1862 to 2457
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   shop_id         5000 non-null   int64         
 1   user_id         5000 non-null   int64         
 2   order_amount    5000 non-null   int64         
 3   total_items     5000 non-null   int64         
 4   payment_method  5000 non-null   object        
 5   created_at      5000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 273.4+ KB


In [56]:
df.tail()

,shop_id,user_id,order_amount,total_items,payment_method,created_at
2630,53,940,112,1,credit_card,2017-03-30 23:12:13
1685,34,818,244,2,cash,2017-03-30 23:16:10
1474,21,815,142,1,cash,2017-03-30 23:26:54
317,52,848,292,2,cash,2017-03-30 23:41:34
2457,95,700,168,1,credit_card,2017-03-30 23:55:35


In [57]:
df['payment_method'].value_counts()

credit_card    1735
debit          1671
cash           1594
Name: payment_method, dtype: int64

In [58]:
df.sort_values(by=['order_amount'])

,shop_id,user_id,order_amount,total_items,payment_method,created_at
4760,92,937,90,1,debit,2017-03-20 07:37:28
1843,92,987,90,1,debit,2017-03-06 07:01:09
2092,92,986,90,1,debit,2017-03-04 06:44:05
4414,92,927,90,1,credit_card,2017-03-17 09:57:01
228,92,757,90,1,debit,2017-03-13 23:57:51
...,...,...,...,...,...,...
4868,42,607,704000,2000,credit_card,2017-03-22 04:00:00
15,42,607,704000,2000,credit_card,2017-03-07 04:00:00
1602,42,607,704000,2000,credit_card,2017-03-17 04:00:00
2969,42,607,704000,2000,credit_card,2017-03-28 04:00:00


In [59]:
# any order amount greater than $10000 came from two different stores
df[df['order_amount'] > 10000]

,shop_id,user_id,order_amount,total_items,payment_method,created_at
4311,78,960,51450,2,debit,2017-03-01 03:02:10
4646,42,607,704000,2000,credit_card,2017-03-02 04:00:00
520,42,607,704000,2000,credit_card,2017-03-02 04:00:00
4412,78,756,51450,2,debit,2017-03-02 04:13:39
4040,78,852,25725,1,cash,2017-03-02 14:31:12
2821,78,814,51450,2,cash,2017-03-02 17:13:25
60,42,607,704000,2000,credit_card,2017-03-04 04:00:00
2492,78,834,102900,4,debit,2017-03-04 04:37:34
4715,78,818,77175,3,debit,2017-03-05 05:10:44
2297,42,607,704000,2000,credit_card,2017-03-07 04:00:00


# Using Interquartile Range to Find Outliers

In [73]:
df.describe()

,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.00000
mean,50.078800,849.092400,3145.128000,8.78720
std,29.006118,87.798982,41282.539349,116.32032
min,1.000000,607.000000,90.000000,1.00000
25%,24.000000,775.000000,163.000000,1.00000
50%,50.000000,849.000000,284.000000,2.00000
75%,75.000000,925.000000,390.000000,3.00000
max,100.000000,999.000000,704000.000000,2000.00000


In [61]:
# Use the 25% and 75& range of the order_amount to calculate outlier
Q1_Order_Amount_Outlier = 163
Q3_Order_Amount_Outlier = 390
IQR_Order_Amount = Q3_Order_Amount_Outlier - Q1_Order_Amount_Outlier

# Calculate the outlier
Minimum_Outlier = Q1_Order_Amount_Outlier - (1.5*IQR_Order_Amount)
Maximum_Outlier = Q3_Order_Amount_Outlier + (1.5*IQR_Order_Amount)

print('Minimum Outlier: ', Minimum_Outlier)
print('Maximum Outlier: ', Maximum_Outlier)

Minimum Outlier:  -177.5
Maximum Outlier:  730.5


In [62]:
# Only need to use Maximum outlier as a condition since minimum outlier is negative
Outlier_Condition = (df['order_amount'] <= Maximum_Outlier)

# create a df without the outliers
df_no_outlier = df[Outlier_Condition]

# sanity check to see if outliers removed
df_no_outlier.sort_values(by=['order_amount']).tail()

,shop_id,user_id,order_amount,total_items,payment_method,created_at
3824,43,736,724,4,credit_card,2017-03-25 17:34:51
2786,43,876,724,4,credit_card,2017-03-01 01:41:43
2312,79,745,724,4,debit,2017-03-27 09:26:31
4659,47,894,725,5,cash,2017-03-01 04:49:59
1124,52,994,730,5,credit_card,2017-03-07 06:54:05


In [63]:
df_no_outlier.describe()

,shop_id,user_id,order_amount,total_items
count,4859.000000,4859.000000,4859.000000,4859.000000
mean,49.852645,849.905742,293.715374,1.950196
std,29.049171,86.887496,144.453395,0.919791
min,1.000000,700.000000,90.000000,1.000000
25%,24.000000,776.000000,162.000000,1.000000
50%,50.000000,850.000000,280.000000,2.000000
75%,74.000000,925.000000,380.000000,3.000000
max,100.000000,999.000000,730.000000,5.000000


# Average Order Value of Each Store

In [64]:
# Total number of shops
print('Total number of shops: ', len(df['shop_id'].value_counts()))

Total number of shops:  100


In [65]:
# sort df by shop number for iteration
df = df.sort_values(by=['shop_id'])

# reset index for iteration
df.reset_index(drop=True, inplace=True)

df.head()

,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,705,474,3,debit,2017-03-13 03:13:36
1,1,934,316,2,debit,2017-03-07 03:29:47
2,1,924,474,3,credit_card,2017-03-16 02:31:40
3,1,951,158,1,credit_card,2017-03-04 20:02:50
4,1,877,474,3,cash,2017-03-09 17:31:51


In [66]:
# empty lists used for iterative purpose
order_amounts = []
item_counts = []

# lists to be used to create new df
shops = []
AOV = []
avg_num_item_sold = []
sum_sales = []
sum_items = []

# iterate shop id to find AOV and average item_counts
for i, val in enumerate(df['shop_id']):
    if val not in shops:
        shops.append(val)
        # if list is not empty (meaning its not the very 1st iteration)
        if order_amounts:
            # append average and sums to average and sum lists
            AOV.append(sum(order_amounts) / len(order_amounts))
            avg_num_item_sold.append(sum(item_counts) / len(item_counts))
            sum_sales.append(sum(order_amounts))
            sum_items.append(sum(item_counts))
            # clear iterative lists for next shop id
            order_amounts.clear()
            item_counts.clear()
            # add the 1st order amount and item count to next shop id iterative list
            order_amounts.append(df['order_amount'][i])
            item_counts.append(df['total_items'][i])
        else:
            # if list is empty (first iteration) it will add first order amount and item count
            order_amounts.append(df['order_amount'][i])
            item_counts.append(df['total_items'][i])
    else:
        # continue to add order amounts and item counts for the same shop id
        order_amounts.append(df['order_amount'][i])
        item_counts.append(df['total_items'][i])

# append values for the final iteration
AOV.append(sum(order_amounts) / len(order_amounts))
avg_num_item_sold.append(sum(item_counts) / len(item_counts))
sum_sales.append(sum(order_amounts))
sum_items.append(sum(item_counts))

In [67]:
# assert that iteration worked correctly
assert (len(AOV) == len(avg_num_item_sold)) & (len(AOV) == len(sum_items))
assert (len(AOV) == len(sum_sales))

In [68]:
# round numbers in lists
AOV_rounded = [round(num, 2) for num in AOV]
avg_num_item_sold_rounded = [round(num, 2) for num in avg_num_item_sold]

In [69]:
# create store_id list
store_id = [num for num in range(1,101)]
len(store_id)

100

In [70]:
# create dictionary before converting to df
store_data = {'AOV': AOV_rounded, 'avg_num_item_sold': avg_num_item_sold_rounded, 
              'sum_sales': sum_sales, 'sum_items': sum_items}

# create df
df_store = pd.DataFrame(data=store_data, index=store_id)
df_store.index.name = 'store_id'

In [71]:
df_store.describe()

,AOV,avg_num_item_sold,sum_sales,sum_items
count,100.000000,100.000000,1.000000e+02,100.000000
mean,3136.834700,8.652600,1.572564e+05,439.360000
std,23935.880965,66.590776,1.216218e+06,3396.366111
min,162.860000,1.730000,6.840000e+03,67.000000
25%,263.672500,1.910000,1.293050e+04,88.000000
50%,308.890000,1.980000,1.488750e+04,100.000000
75%,336.627500,2.080000,1.760000e+04,111.250000
max,235101.490000,667.900000,1.199018e+07,34063.000000


In [90]:
df_store.head()

,AOV,avg_num_item_sold,sum_sales,sum_items
store_id,,,,
1,308.82,1.95,13588,86
2,174.33,1.85,9588,102
3,305.25,2.06,14652,99
4,258.51,2.02,13184,103
5,290.31,2.04,13064,92


In [92]:
''' Create shopify_pre_store_aov csv'''
# df_store.to_csv('shopify_per_store_aov')

'''confirm that csv got created correctly'''
df_confirm = pd.read_csv('shopify_per_store_aov')
df_confirm.head()

# Deeper Dive into Outliers

In [81]:
'''deep dive into shop 42 user 607 who has huge money'''
# df[df['shop_id']==42]
df[df['user_id']==607].sort_values('created_at')

,shop_id,user_id,order_amount,total_items,payment_method,created_at
2092,42,607,704000,2000,credit_card,2017-03-02 04:00:00
2093,42,607,704000,2000,credit_card,2017-03-02 04:00:00
2104,42,607,704000,2000,credit_card,2017-03-04 04:00:00
2100,42,607,704000,2000,credit_card,2017-03-07 04:00:00
2099,42,607,704000,2000,credit_card,2017-03-07 04:00:00
2126,42,607,704000,2000,credit_card,2017-03-11 04:00:00
2089,42,607,704000,2000,credit_card,2017-03-12 04:00:00
2109,42,607,704000,2000,credit_card,2017-03-15 04:00:00
2121,42,607,704000,2000,credit_card,2017-03-17 04:00:00
2097,42,607,704000,2000,credit_card,2017-03-19 04:00:00


In [88]:
'''deep dive into shop 78 user 607 who has huge money'''
df_shop_78 = df[df['shop_id']==78].sort_values('user_id')
df_shop_78.head()

,shop_id,user_id,order_amount,total_items,payment_method,created_at
3865,78,707,51450,2,cash,2017-03-26 04:38:52
3870,78,709,51450,2,cash,2017-03-27 11:04:04
3883,78,740,25725,1,debit,2017-03-12 20:10:58
3888,78,745,25725,1,credit_card,2017-03-18 13:13:07
3901,78,756,51450,2,debit,2017-03-02 04:13:39


In [89]:
# find how many items shop78 sold
sum(df_shop_78['total_items'])

88